# Pipelines subpackage

The `opr.pipelines` subpackage contains ready-to-use pipelines for model inference.

## Usage example - Place Recognition Pipeline

In [1]:
from hydra.utils import instantiate
import numpy as np
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation

from opr.datasets.itlp import ITLPCampus
from opr.pipelines.place_recognition import PlaceRecognitionPipeline

### Config

In [2]:
DATABASE_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/outdoor/old/00_2023-02-10"
QUERY_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/outdoor/old/01_2023-02-21"

DEVICE = "cuda"

MODEL_CONFIG_PATH = "../configs/model/place_recognition/minkloc3d.yaml"
WEIGHTS_PATH = "../weights/place_recognition/minkloc3d_nclt.pth"

#### Functions

In [3]:
def pose_to_matrix(pose):
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error

#### Init query dataset

The pipeline infer method accepts an input in the format of dictionary with keys in the following format:
- `"image_{camera_name}"` for images from cameras,
- `"mask_{camera_name}"` for semantic segmentation masks,
- `"pointcloud_lidar_coords"` for pointcloud coordinates from lidar,
- `"pointcloud_lidar_feats"` for pointcloud features from lidar.

The data type of all values are `torch.Tensor`.

You can load and preprocess the data manually, but in this example we will use the `opr.datasets.itlp.ITLPCampus` ` dataset class. 

In [4]:
query_dataset = ITLPCampus(
    dataset_root=QUERY_TRACK_DIR,
    sensors=["lidar"],
    mink_quantization_size=0.5,
    load_semantics=False,
    load_text_descriptions=False,
    load_text_labels=False,
    load_aruco_labels=False,
    indoor=False,
)


#### Initialize model

We will use hydra's `instantiate` function to initialize the model. The model is a `MinkLoc3D` - a simple LiDAR-only architecture.

In [5]:
model_config = OmegaConf.load(MODEL_CONFIG_PATH)
model = instantiate(model_config)

#### Initialize pipeline

The minimum requirement to initialize the `PlaceRecognitionPipeline` is that the database directory should contain the `index.faiss` file and the `track.csv` file.

The `index.faiss` file is a Faiss index, which contains the descriptors of the database. The `track.csv` file contains the metadata of the database, including the id and the pose of the descriptors.

The details on how to create the database are described in the [build_database.ipynb](./build_database.ipynb) notebook.

Note that the actual data are not required, as the pipeline will only load the `index.faiss` and the `track.csv` file. This can be useful in the real-world scenario, where the database size is too large to be stored on the local machine.

In [6]:
pipe = PlaceRecognitionPipeline(
    database_dir=DATABASE_TRACK_DIR,
    model=model,
    model_weights_path=WEIGHTS_PATH,
    device=DEVICE,
)


#### Run inference

In [7]:
sample_data = query_dataset[5]
sample_pose_gt = sample_data.pop("pose")  # removing those keys are not necessary, we just
sample_data.pop("idx")                    # want to simulate that we pass the data without GT information :)
print(f"sample_data.keys() = {sample_data.keys()}")
sample_output = pipe.infer(sample_data)
print(f"sample_output.keys() = {sample_output.keys()}")
print(f"sample_output['idx'] = {sample_output['idx']}")
print(f"pose = {sample_output['pose']}")
print(f"pose_gt = {sample_pose_gt.numpy()}")
dist_error, angle_error = compute_error(sample_output["pose"], sample_pose_gt.numpy())
print(f"dist_error = {dist_error}, angle_error = {angle_error}")


sample_data.keys() = dict_keys(['pointcloud_lidar_coords', 'pointcloud_lidar_feats'])
sample_output.keys() = dict_keys(['idx', 'pose', 'descriptor'])
sample_output['idx'] = 6
pose = [ 6.40806188e-01 -6.96985360e+00 -2.62666965e+00  2.23428939e-03
  1.21724469e-02  3.57296189e-01  9.33909135e-01]
pose_gt = [-0.3476145  -7.5303183  -1.8204867  -0.01149435  0.01080806  0.3418056
  0.9396382 ]
dist_error = 1.3932074823071565, angle_error = 2.466076215796548
